In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import matplotlib.pyplot as plt
import os

from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score

np.random.seed(252525)
torch.manual_seed(252525)

from hugeica import *

from data.cifar_novelty import Cifar10_OneClass


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
net = models.efficientnet_b4(pretrained=True).features[:4]
net = net.to(device)
net.eval()

In [9]:
p_sizes = [26, 26, 22, 26, 24, 26, 22, 24, 26, 26]
s = 2

log = []
for clazz in range(0, 10):

    test_classes = [0,1,2,3,4,5,6,7,8,9]
    test_classes.remove(clazz)
    trans = [transforms.ToTensor()]
    p = p_sizes[clazz]
    p = 26
    
    np.random.seed(32)
    X_, X_valid_, X_test_ = Cifar10_OneClass(train_classes=[clazz], test_classes=test_classes, z_normalize=False, balance=True, transform=transforms.Compose(trans))[0]
    X, X_valid, X_test = preprocess(X_, X_valid_, X_test_, norm_contrast=True, DC=True, channels=None)[:3]
    X = X.reshape(len(X), 3, 32, 32)
    X_valid = X_valid.reshape((len(X_valid), 3, 32, 32))
    X_test = X_test.reshape((len(X_test), 3, 32, 32))
    
    c = X.shape[1]
    t = X.shape[2]
    n = len(X_valid) + len(X_test)
        
    #with torch.no_grad():
    #    X = torch.cat([ net(torch.from_numpy( X_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_), bs)]).cpu().numpy()
    #    X_valid = torch.cat([ net(torch.from_numpy( X_valid_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_valid_), bs)]).cpu().numpy()
    #    X_test = torch.cat([ net(torch.from_numpy( X_test_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_test_), bs)]).cpu().numpy()

    model = SFA(shape=(c, t, t), 
                    BSZ=(p, p), 
                    stride=s, 
                    n_components="q9999",
                    remove_components=0,
                    max_components=5000,
                    min_components=100,
                    mode="ta")

    model.fit(X, 1, bs=2000, lr=1e-3, logging=-10)
    
    auc_g, tg       = auc_global_mean_shift(model, X_valid, X_test)
    auc_l, tl       = auc_local_mean_shift(model, X, X_valid, X_test, size=2)
    auc_c, tc, cset = auc_cluster_mean_shift(model, X, X_valid, X_test, True, return_coreset=True)
    auc_cc, tcc     = auc_cluster_mean_shift(model, X, X_valid, X_test, False, k=len(cset))
    auc_s, ts       = auc_sources_mean_shift(model, X, X_valid, X_test)
    
    df = pd.DataFrame(np.asarray([clazz, auc_g, auc_l, auc_c, auc_cc, auc_s, n/tg, n/tl, n/tc, n/tcc, n/ts, p, t, model.model.n_components])[:, None].T, 
                      columns=["class", "AUC-g", "AUC-l", "AUC-c", "AUC-cc", "AUC-s", "fps-g", "fps-l", "fps-c", "fps-cc", "fps-s", "p_size", "t", "n_components"]) 
    
    log.append(df)
    
    display(pd.concat(log))
    

Files already downloaded and verified
Files already downloaded and verified
Cifar10_OneClass(z_normalize=False, train_classes=[0], test_classes=[1, 2, 3, 4, 5, 6, 7, 8, 9], data_train=(5000, 3072), data_test_inliers=(1000, 3072), data_test_outliers=(1000, 3072))
Getting transformed features...
Using flat_X as features.
Maximum distance from cluster centers is 88.31
n_clusters 800
n_clusters 800


,class,AUC-g,AUC-l,AUC-c,AUC-cc,AUC-s,fps-g,fps-l,fps-c,fps-cc,fps-s,p_size,t,n_components
0,0.0,0.65168,0.616702,0.586025,0.584572,0.650238,1727.66992,1368.670805,1182.313211,1179.636075,1735.791103,26.0,32.0,1581.0


Files already downloaded and verified
Files already downloaded and verified
Cifar10_OneClass(z_normalize=False, train_classes=[1], test_classes=[0, 2, 3, 4, 5, 6, 7, 8, 9], data_train=(5000, 3072), data_test_inliers=(1000, 3072), data_test_outliers=(1000, 3072))


/home/matthias/Desktop/workspace/fasterica/hugeica/SFA.py:78: UserWarning: n_components=2028 > np.min([max_components=2000,bs=2000]). Setting n_components=2000
  warnings.warn(f"n_components={n_components} > np.min([max_components={max_components},bs={bs}]). Setting n_components={np.min([max_components, bs])}")


Getting transformed features...
Using flat_X as features.
Maximum distance from cluster centers is 93.08
n_clusters 800
n_clusters 800


...

,class,AUC-g,AUC-l,AUC-c,AUC-cc,AUC-s,fps-g,fps-l,fps-c,fps-cc,fps-s,p_size,t,n_components
0,0.0,0.651680,0.616702,0.586025,0.584572,0.650238,1727.669920,1368.670805,1182.313211,1179.636075,1735.791103,26.0,32.0,1581.0
0,1.0,0.528927,0.488555,0.481010,0.477767,0.529692,1733.190275,1384.008665,1195.695239,1195.213452,1714.505282,26.0,32.0,1616.0
0,2.0,0.497051,0.475712,0.452444,0.451765,0.502848,1722.772798,1370.338280,1162.866922,1172.332062,1675.659199,26.0,32.0,1640.0
0,3.0,0.659156,0.647488,0.629964,0.629120,0.657026,1888.344952,1476.395315,1277.248894,1273.410466,1858.548727,26.0,32.0,1527.0
0,4.0,0.459349,0.450935,0.440673,0.441194,0.457125,1690.924337,1335.865157,1155.741399,1173.111788,1650.407683,26.0,32.0,1629.0
0,5.0,0.705355,0.685072,0.658474,0.658052,0.703681,1858.711803,1470.167473,1266.815347,1279.420185,1805.891499,26.0,32.0,1517.0
0,6.0,0.528417,0.470659,0.411253,0.411059,0.523927,1654.548653,1281.703827,1144.246133,1124.967479,1630.387854,26.0,32.0,1638.0
0,7.0,0.595815,0.573340,0.556630,0.557495,0.594413,1848.873686,1487.615892,1276.380964,1271.246601,1837.654107,26.0,32.0,1525.0
0,8.0,0.717013,0.718158,0.713390,0.712054,0.715194,1801.119542,1474.881709,1274.169647,1269.443259,1815.994706,26.0,32.0,1518.0


Files already downloaded and verified
Files already downloaded and verified
Cifar10_OneClass(z_normalize=False, train_classes=[9], test_classes=[0, 1, 2, 3, 4, 5, 6, 7, 8], data_train=(5000, 3072), data_test_inliers=(1000, 3072), data_test_outliers=(1000, 3072))


/home/matthias/Desktop/workspace/fasterica/hugeica/SFA.py:78: UserWarning: n_components=2028 > np.min([max_components=2000,bs=2000]). Setting n_components=2000
  warnings.warn(f"n_components={n_components} > np.min([max_components={max_components},bs={bs}]). Setting n_components={np.min([max_components, bs])}")


Getting transformed features...
Using flat_X as features.
Maximum distance from cluster centers is 91.58
n_clusters 800
n_clusters 800


,class,AUC-g,AUC-l,AUC-c,AUC-cc,AUC-s,fps-g,fps-l,fps-c,fps-cc,fps-s,p_size,t,n_components
0,0.0,0.651680,0.616702,0.586025,0.584572,0.650238,1727.669920,1368.670805,1182.313211,1179.636075,1735.791103,26.0,32.0,1581.0
0,1.0,0.528927,0.488555,0.481010,0.477767,0.529692,1733.190275,1384.008665,1195.695239,1195.213452,1714.505282,26.0,32.0,1616.0
0,2.0,0.497051,0.475712,0.452444,0.451765,0.502848,1722.772798,1370.338280,1162.866922,1172.332062,1675.659199,26.0,32.0,1640.0
0,3.0,0.659156,0.647488,0.629964,0.629120,0.657026,1888.344952,1476.395315,1277.248894,1273.410466,1858.548727,26.0,32.0,1527.0
0,4.0,0.459349,0.450935,0.440673,0.441194,0.457125,1690.924337,1335.865157,1155.741399,1173.111788,1650.407683,26.0,32.0,1629.0
0,5.0,0.705355,0.685072,0.658474,0.658052,0.703681,1858.711803,1470.167473,1266.815347,1279.420185,1805.891499,26.0,32.0,1517.0
0,6.0,0.528417,0.470659,0.411253,0.411059,0.523927,1654.548653,1281.703827,1144.246133,1124.967479,1630.387854,26.0,32.0,1638.0
0,7.0,0.595815,0.573340,0.556630,0.557495,0.594413,1848.873686,1487.615892,1276.380964,1271.246601,1837.654107,26.0,32.0,1525.0
0,8.0,0.717013,0.718158,0.713390,0.712054,0.715194,1801.119542,1474.881709,1274.169647,1269.443259,1815.994706,26.0,32.0,1518.0
0,9.0,0.575735,0.545267,0.555492,0.553296,0.568951,1766.394237,1388.098924,1221.006189,1199.318834,1738.987882,26.0,32.0,1592.0


In [ ]:
bs = 50
t = 14 # tile_size
p = 4 # fallback patch_size
s = 1 # stride

net = models.efficientnet_b4(pretrained=True).features[:6]
net = net.to(device)
net.eval()

log = []

for clazz in range(0, 15):
    
    X_, X_valid_, X_test_, X_labels_, T = dataloader(clazz, P=224, s=224, label_per_patch=False, load_size=256, crop_size=224, normalize=False)

    with torch.no_grad():
        X = torch.cat([ net(torch.from_numpy( X_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_), bs)]).cpu().numpy()
        X_valid = torch.cat([ net(torch.from_numpy( X_valid_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_valid_), bs)]).cpu().numpy()
        X_test = torch.cat([ net(torch.from_numpy( X_test_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_test_), bs)]).cpu().numpy()

    shapes = (X.shape, X_test.shape, X_valid.shape)
    c = X.shape[1]
    tt = 0.
    n = len(X_valid) + len(X_test)
    print(shapes)
    
    model = SFA(shape=(c, t, t), 
                    BSZ=(p, p), 
                    stride=s, 
                    n_components="q9999",
                    remove_components=0,
                    max_components=5000,
                    min_components=100,
                    mode="ta")

    model.fit(X, 1, bs=3000, lr=1e-3, logging=-10)
    
    auc_g, tg       = auc_global_mean_shift(model, X_valid, X_test)
    auc_l, tl       = auc_local_mean_shift(model, X, X_valid, X_test, size=2)
    auc_c, tc, cset = auc_cluster_mean_shift(model, X, X_valid, X_test, True, return_coreset=True)
    auc_cc, tcc     = auc_cluster_mean_shift(model, X, X_valid, X_test, False, k=len(cset))
    auc_s, ts       = auc_sources_mean_shift(model, X, X_valid, X_test)
    #auc_cs, tcs     = auc_coreset(X, X_valid, X_test)

    df = pd.DataFrame(np.asarray([clazz, auc_g, auc_l, auc_c, auc_cc, auc_s, n/tg, n/tl, n/tc, n/tcc, n/ts, p, t, model.model.n_components])[:, None].T, 
                      columns=["class", "AUC-g", "AUC-l", "AUC-c", "AUC-cc", "AUC-s", "fps-g", "fps-l", "fps-c", "fps-cc", "fps-s", "p_size", "t", "n_components"]) 
    log.append(df)
    
    display(pd.concat(log))
    
pd.DataFrame(np.asarray([pd.concat(log)["AUC-g"].mean(), pd.concat(log)["AUC-l"].mean(), pd.concat(log)["AUC-c"].mean(), pd.concat(log)["AUC-cc"].mean(), pd.concat(log)["AUC-s"].mean()])[:, None].T, 
             columns=["Mean-g", "Mean-l", "Mean-c", "Mean-cc", "Mean-s"])

In [ ]:
bs = 50
p = 4 # fallback patch_size
s = 1 # stride

net = mods.efficientnet_b4(pretrained=True).features
net = net.to(device)
net.eval()

for clazz in [13]:
    
    X_, X_valid_, X_test_, X_labels_, T = dataloader(clazz, P=224, s=224, label_per_patch=False, load_size=256, crop_size=224, normalize=False)

    X = concat_features(X_, net, layers=[5,6], fmap_pool=False)
    X_valid = concat_features(X_valid_, net, layers=[5,6], fmap_pool=False)
    X_test = concat_features(X_test_, net, layers=[5,6], fmap_pool=False)

    shapes = (X.shape, X_test.shape, X_valid.shape)
    c = X.shape[1]
    t = X.shape[2] # tile_size
    n = len(X_valid) + len(X_test)
    print(shapes)
    
    model = SFA(shape=(c, t, t), 
                    BSZ=(p, p), 
                    stride=s, 
                    n_components="q9999",
                    remove_components=0,
                    max_components=5000,
                    min_components=100,
                    mode="ta")

    model.fit(X, 1, bs=5000, lr=1e-3, logging=-10)
    
    auc_g, tg       = auc_global_mean_shift(model, X_valid, X_test)
    auc_l, tl       = auc_local_mean_shift(model, X, X_valid, X_test, size=2)
    auc_c, tc, cset = auc_cluster_mean_shift(model, X, X_valid, X_test, True, return_coreset=True)
    auc_cc, tcc     = auc_cluster_mean_shift(model, X, X_valid, X_test, False, k=len(cset))
    auc_s, ts       = auc_sources_mean_shift(model, X, X_valid, X_test)
    #auc_cs, tcs     = auc_coreset(X, X_valid, X_test)

    df = pd.DataFrame(np.asarray([clazz, auc_g, auc_l, auc_c, auc_cc, auc_s, n/tg, n/tl, n/tc, n/tcc, n/ts, p, t, model.model.n_components])[:, None].T, 
                      columns=["class", "AUC-g", "AUC-l", "AUC-c", "AUC-cc", "AUC-s", "fps-g", "fps-l", "fps-c", "fps-cc", "fps-s", "p_size", "t", "n_components"]) 
    
    plt.rcParams["figure.figsize"] = (25, 25)
    map_i, map_o = local_mean_shift(model, X, X_valid, X_test, size=2)
    anomaly_map = make_anomaly_map(map_i, map_o)[1]
    anomaly_map = anomaly_map + X_test_images
    show( torchvision.utils.make_grid(torch.from_numpy(anomaly_map), nrow=10, normalize=True,  value_range=(0,  1), rscale_each=False , padding=3 ) )